In [ ]:
from datenguidepy.query_builder import Query
from datenguidepy.query_execution import QueryExecutioner

import pandas as pd
from functools import partial

In [ ]:
# from unittest.mock import Mock

In [ ]:
%%time

def get_all_regions():
    def nuts_query(nuts_level):
        q = Query.all_regions(nuts=nuts_level)
        return q

    def lau_query(lau_level):
        q = Query.all_regions(lau = lau_level)
        return q

    qb_all = Query.all_regions()

    qe = QueryExecutioner()
    all_regions = qe.run_query(qb_all)
    r_nuts1 = qe.run_query(nuts_query(1))
    r_nuts2 = qe.run_query(nuts_query(2))
    r_nuts3 = qe.run_query(nuts_query(3))
    r_lau1 = qe.run_query(lau_query(1))
    r_lau2 = qe.run_query(lau_query(2))

    levels = {
    'nuts1':r_nuts1,
    'nuts2':r_nuts2,
    'nuts3':r_nuts3,
    'lau':r_lau1,
    # 'lau2':r_lau2
    }

    def isAnscestor(region_id,candidate):
        return region_id.startswith(candidate) and candidate != region_id


    def parent(region_id,region_details):
        desc =  region_details.assign(ansc = lambda df:df.index.map(lambda i: isAnscestor(region_id,i))).query('ansc')
        max_lev = desc.level.max()
        parent_frame = desc.query('level == @max_lev')
        if not parent_frame.empty:
            return parent_frame.iloc[0,:].name
        else:
            None

    all_regions_df = pd.concat([pd.DataFrame(page['data']['allRegions']['regions']) for page in all_regions[0].query_results]).set_index('id')

    level_df = pd.concat(pd.concat([pd.DataFrame(page['data']['allRegions']['regions']) for page in levels[k][0].query_results]).assign(level = k) for k in levels)

    all_rg_parents= (all_regions_df.join(
        level_df.set_index('id').loc[:,'level']
        ).assign(parent = lambda df: df.index.map(partial(parent,region_details = all_regions_df.assign(level = lambda df:df.index.map(len)))))
    )
    all_rg_parents.loc[all_rg_parents.level == 'nuts1','parent'] = 'DG'
    
    return all_rg_parents

In [ ]:
qb_all.get_graphql_query()

In [ ]:
all_regions_df.head()

In [ ]:
ar_detail.head()

In [ ]:
all_rg_parents.head()

In [ ]:
level_df.head()

In [ ]:
    def isDescendent(region_id,candidate):
        return candidate.startswith(region_id) and candidate != region_id

    def children(region_id,region_details):
        desc =  region_details.assign(desc = lambda df:df.index.map(lambda i: isDescendent(region_id,i))).query('desc')
        min_lev = desc.level.min()
        return desc.query('level == @min_lev')


In [ ]:
all_regions_df.join(level_df.set_index('id').loc[:,'level']).to_csv('regions.csv')

In [ ]:
level_df.level.value_counts(ascending=True)

This does not correspond to the current german nuts. These changed in the past however. One shoudl try to make a query with statistics from 2018 or later and do the same analysis again, to see whether one gets an up to date view of the nuts.

In [ ]:
level_df.id.str.len().value_counts(ascending=True)

In [ ]:
level_df.groupby('level').apply(lambda df: (df.id.str.len().min(),df.id.str.len().max()))

In [ ]:
no_level = all_regions_df.merge(level_df,on='id',how='left').query('level.isna()')
print(no_level.shape)
no_level.head()

In [ ]:
all_rg_parents.to_csv('regions.csv')

In [ ]:
def hirachy_up(lowestids,hirachy_frame = all_rg_parents):
    anscestors = []
    current_ids = lowestids
    while len(current_ids)>0:
        current_regions = hirachy_frame.query('index.isin(@current_ids)')
        anscestors.append(current_regions)
        current_ids = current_regions.dropna().parent.unique()
    return pd.concat(anscestors).sort_index()

def hirachy_down(highest_ids,lowest_level='lau',hirachy_frame = all_rg_parents):
    descendents = [hirachy_frame.query('index.isin(@highest_ids)')]
    current_ids = highest_ids
    while len(current_ids)>0:
        current_regions = hirachy_frame.query('parent.isin(@current_ids)')
        descendents.append(current_regions)
        current_ids = current_regions.dropna().index.unique()
#         print(current_regions)
#         print(current_regions.level.unique())
        if lowest_level in current_regions.level.unique():
            break
    return pd.concat(descendents).sort_index()

def siblings(region_id,hirachy_frame = all_rg_parents):
    parent = hirachy_frame.query('index == @region_id').loc[:,'parent'].iloc[0]
    return hirachy_frame.query('parent == @parent')

In [ ]:
siblings('051')

In [ ]:
hirachy_up(['051']).sort_values(['parent','level'])

dg_map = lambda x: x if x != 'DG' else ''

hu = hirachy_up(['051'])
hu.assign(sort_col = lambda df: df.index.map(dg_map)).sort_values('sort_col').drop('sort_col',axis=1)

In [ ]:
hirachy_down(['05'],None)

In [ ]:
ar_detail.query('id == "05911"')

In [ ]:
parent_rel = ar_detail.reset_index().assign(parent = lambda df: df.id.map(parent))

In [ ]:
all_rg_parents.loc[all_rg_parents.level == 'nuts1','parent'] = 'DG'

In [ ]:
def children(region_id):
    return parent_rel.query('parent == @region_id')

In [ ]:
children('16')